In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as pl
from bson.json_util import dumps

#The Intention of this file is to provide an interface for the AirBnB dataset. Using property type,
# we Organize the data and present it visually in a data table and chart. 

#importing crud operations python file
from retalListing import retalListing


#Using credentials specific to my Atlas cluster/mongo server
username = "mmcneeley"
password = "snhuCapstone"
rentalListing = rentalListing(username, password)


# class read method must support return of cursor object
df = pd.DataFrame.from_records(listing.read({}))


#Creating & Loading Dashboard named 'AirBnb'
app = JupyterDash('AirBnB')



app.layout = html.Div([
    #    html.Div(id='hidden-div', style={'display':'none'}),
     html.Center(html.B(html.H1('Madeline McNeeley Capstone Project Dec 2022'))),
    
    #new datatable, with dropdown options
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': 'Apartment', 'value': 'apartment'},
                {'label': 'House', 'value': 'house'},
                {'label': 'Guest House', 'value': 'guest'},
                {'label': 'Reset', 'value': 'reset'} 
            ],
            value='House'
        ),
        html.Div(id='dd-output-container')
    ]),
    #populating table with rental listings
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        # filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    

    html.Br(),
    html.Hr(),
    #Sets up the dashboard so that the chart is centered 
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 dcc.Graph(
                     id="pie-chart",
                     className='col s12 m6',
                 )
             ])
])


#drop down option values. When drop down option is picked, data table will only load data of that specific proprty_type

@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    
    #When user selects apartment, only apartment listings are populated in the chart & data table
    if (value =='apartment'):
        df = pd.DataFrame(list(listings.read(
            { 
            "property_type": {
                "$in": [
                    "Apartment"
                ]
            }
        }
        )))
        
        #When user selects house, only house listings are populated in the chart & data table
    elif (value == 'house'):
        df = pd.DataFrame(list(listings.read(
            { 
            "property_type": {
                "$in": [
                    "House"
                ]
            }
        }
        )))
        
        #When user selects guest house, only guest house listings are populated in the chart & data table
    elif (value == 'guest'):
        df = pd.DataFrame(list(listings.read(
            { 
            "property_type": {
                "$in": [
                    "Guest House"
                ]
            }
        }
        )))
    else:
        
        #resets listing if user clicks the reset option in dropdown
        df = pd.DataFrame.from_records(listings.read({}))
    return df.to_dict('listings')
    

#app call back is used to generate everything when page is loaded. 
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
#genereates data table rows and columns
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
#genereates pie chart to show percentage property types.
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(
        dff,
        names='property_type',
    )
    return fig


app